## JAZMIN VIVIANA SANTOYO VEGA
###**Sistema de Comunicación de un Robot Espacial**
####**Teoría de la Información**

### Librerías

In [27]:
import random
import pickle
import math
import matplotlib.pyplot as plt
from collections import Counter
import heapq
from collections import defaultdict
import pandas as pd
import struct
import hashlib

In [28]:
# Aquí la fuente de información son los sensores del robot recopilan datos
def recopilar_datos():
  # Simulacion de algunos atributos que recopila el sensor del robot
  objetos = ["mineral, roca & meteorito", "roca, cristal, agua & hielo", "cristal, regolito, agua", "hielo, relieves, regolito & meteorito", "agua, meterorito & relieves","objeto no identificado"]
  ubicacion = (round(random.uniform(-1000, 1000), 2), round(random.uniform(-1000, 1000), 2), round(random.uniform(-1000, 1000), 2))
  imagenes = ["imagen1.jpg", "imagen2.jpg"]

  objeto = random.choice(objetos)
  datos_recopilados = {
      "Se detectó": objeto,
      "Ubicación": ubicacion,
      "Imagenes": imagenes
  }
  return datos_recopilados

### Calcular frecuencia

In [29]:
# recibe una cadena de texto o de numeros para calcular la frecuencia y devolver a la codificación que se desee
def obtiene_frecuencia(cadena):
    frecuencias = dict(Counter(cadena))
    return frecuencias

###Codificación Tunstall

In [30]:
def codificacion_tunstall(cadena):

    frecuencias = obtiene_frecuencia(cadena)

    # creo una lista de tuplas (símbolo, frecuencia) ordenadas por frecuencia
    simbolos_frecuencias = sorted(frecuencias.items(), key=lambda x: x[1], reverse=True)

    # diccionario para asignar códigos Tunstall a cada símbolo
    codigos_tunstall = {}
    num_bits = 0

    while len(simbolos_frecuencias) > 2 ** num_bits:
        num_bits += 1

    for i, (simbolo, _) in enumerate(simbolos_frecuencias):
        codigo = format(i, f'0{num_bits}b')
        codigos_tunstall[simbolo] = codigo

    # la cadena de entrada utilizando los códigos Tunstall
    cadena_codificada_t = " ".join([codigos_tunstall[s] for s in cadena])

    return cadena_codificada_t, codigos_tunstall

### Función que hashea los paquetes

In [31]:
def hashear_paquetes(cadena_codificada_t):
    """Genera un hash para cada grupo de cuatro bits separados por un espacio."""
    # elimino espacios y divido la cadena en grupos de cuatro bits
    grupos_cuatro_bits = cadena_codificada_t.replace(" ", "")
    grupos_cuatro_bits = [grupos_cuatro_bits[i:i+4] for i in range(0, len(grupos_cuatro_bits), 4)]
    # genero el hash para cada grupo de cuatro bits y almaceno en una lista
    paquete_hashes = []

    for grupo in grupos_cuatro_bits:
        hash_grupo = hashlib.sha256(grupo.encode()).hexdigest()
        paquete_hashes.append(hash_grupo)

    return paquete_hashes

###Transmisor

* Imprimo los paquetes en código tunstall solo para visualizar pero estos no se transmiten.

* Codifico los paquetes en Tunstall

* Hasheo los paquetes.

In [32]:
def transmisor_de_la_nave_espacial(datos_recopilados):
    objeto_detectado = datos_recopilados["Se detectó"]
    ubicacion = datos_recopilados["Ubicación"]
    imagenes = datos_recopilados["Imagenes"]

    # CODIFICACIÓN TUNSTALL
    paquete_objeto_codificado, codigos_objeto = codificacion_tunstall(objeto_detectado)
    paquete_ubicacion_codificado, codigos_ubicacion = codificacion_tunstall(str(ubicacion))

    paquetes_codificados = {
        "Se detectó": paquete_objeto_codificado,
        "Ubicación": paquete_ubicacion_codificado,
        "Imagenes": imagenes
    }
    print("\nPaquetes codificados en Tunstall:")
    print(paquetes_codificados)

    handshake_codigos_objeto = pd.DataFrame(list(codigos_objeto.items()), columns=['Simbolo', 'Código'])
    handshake_codigos_ubicacion = pd.DataFrame(list(codigos_ubicacion.items()), columns=['Simbolo', 'Código'])

    # Genero los hashes para los paquetes codificados
    paquete_objeto_hasheado = hashear_paquetes(paquete_objeto_codificado)
    paquete_ubicacion_hasheado = hashear_paquetes(paquete_ubicacion_codificado)

    datos_transmitidos_hasheados = {
        "Se detectó": paquete_objeto_hasheado,
        "Ubicación": paquete_ubicacion_hasheado,
        "Imagenes": imagenes
    }

    return datos_transmitidos_hasheados, codigos_objeto, codigos_ubicacion

###Canal

* En el canal viajan los paquetes hasheados.

In [33]:
# Este es el medio por el que viajaran los datos transmitidos
# esta función simula el canal, en este caso es por medio de fibra óptica
def fibra_optica(datos_transmitidos_hasheados):

    """ DETALLES DE LA FIBRA ÓPTICA """
    # longitud de la fibra en km
    longitud_km = 15
    # velocidad de la luz en ms
    velocidad_ms = 7000
    # longitud de la fibra a metros
    longitud_m = longitud_km * 1000
    # tiempo de propagación en segundos
    tiempo = longitud_m / velocidad_ms

    objeto_codificado = datos_transmitidos_hasheados["Se detectó"]
    ubicacion_codificada = datos_transmitidos_hasheados["Ubicación"]
    imagenes_codificada = datos_transmitidos_hasheados["Imagenes"]
    datos_canal = datos_transmitidos_hasheados.copy()

    datos_canal["Se detectó"] = objeto_codificado
    datos_canal["Ubicación"] = ubicacion_codificada

    return datos_canal, tiempo

###Decodificador del código Tunstall

In [34]:
def decodificacion_tunstall(cadena_codificada_t, codigos_tunstall):
    # recibe los paquetes en tipo de variable lista así que lo vuelvo cadena
    cadena_codificada_t = [item for sublist in cadena_codificada_t for item in sublist]
    cadena_codificada_t = "".join(map(str, cadena_codificada_t))
    cadena_tunstall_decodificada = ""
    codigo_actual = ""

    for bit in cadena_codificada_t:
        codigo_actual += bit
        simbolo_encontrado = None

        for simbolo, codigo in codigos_tunstall.items():
            if codigo_actual.startswith(codigo):
                simbolo_encontrado = simbolo
                break

        if simbolo_encontrado is not None:
            cadena_tunstall_decodificada += simbolo_encontrado
            codigo_actual = codigo_actual[len(codigo):]

    return cadena_tunstall_decodificada

###Función que hashea los códigos Tunstall del handshake

In [35]:
def hashear_handshake(codigos_tunstall):
    df_codigos = pd.DataFrame(list(codigos_tunstall.items()), columns=['Simbolo', 'Código'])
    df_codigos['Hash'] = df_codigos['Código'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())
    # este diccionario mapea cada código a su hash correspondiente
    diccionario_hash = dict(zip(df_codigos['Código'], df_codigos['Hash']))
    return diccionario_hash

###Búsqueda binaria
Esta función recibe los handshake y los paquetes hasheados.

* Uso la recursividad para  dividir la lista a la mitad en cada llamada hasta que se encuentre el paquete hasheado que se busca en la lista de hash en el handshake o determine que no se encuentra.

* Si el paquete se encuentra en el handshake, retorna que el paquete se encontró y va guardando en una lista el código tunstall al que corresponde.

* Devuelve que se encontraron todos los paquetes y los paquetes codificados en tunstall. Por lo que se recontruye el mensaje original codificado en Tunstall.

In [36]:
def busqueda_binaria(lista_handshake, paquetes_hashes):
    # convierto la los paquetes hasheados en una lista
    hashes_a_buscar = paquetes_hashes
    lista_cods_tunstall = [fila[0] for fila in lista_handshake]
    lista_hashes_handshake = [fila[1] for fila in lista_handshake]

    # lista de hashes ordenada para realizar la búsqueda binaria
    lista_hashes_ordenados = sorted(enumerate(lista_hashes_handshake), key=lambda x: x[1])

    # lista para almacenar los códigos correspondientes a los hashes encontrados
    codigos_encontrados = []

    # Con esta función aplico recursividad para hacer la busqueda
    def busqueda_binaria_recursiva(inicio, fin, hash_a_buscar):
        if inicio > fin:
            return None
        medio = (inicio + fin) // 2
        if lista_hashes_ordenados[medio][1] == hash_a_buscar:
            indice_encontrado = lista_hashes_ordenados[medio][0]
            codigo_tunstall_encontrado = lista_cods_tunstall[indice_encontrado]
            codigos_encontrados.append(codigo_tunstall_encontrado)
            #print(f"Hash encontrado: {hash_a_buscar}, Código Tunstall correspondiente: {codigo_tunstall_encontrado}")
            return True
        elif lista_hashes_ordenados[medio][1] < hash_a_buscar:
            return busqueda_binaria_recursiva(medio + 1, fin, hash_a_buscar)
        else:
            return busqueda_binaria_recursiva(inicio, medio - 1, hash_a_buscar)

    # itera sobre cada hash a buscar y realiza la búsqueda binaria
    # si todos los hashes se encuentran, retorna True y la lista de códigos encontrados
    for hash_a_buscar in hashes_a_buscar:
        busqueda_binaria_recursiva(0, len(lista_hashes_ordenados) - 1, hash_a_buscar)

    return True, codigos_encontrados

###Receptor
* El receptor recibe el hanshake y hashea los códigos Tunstall.

* Manda a llamar la función de búsqueda binaria para encontrar los paquetes en el handshake.

* Reconstruye el mensaje de los paquetes codificados en tunstall para la decodificación.

* Notifica si se encontraron todos los paquetes hasheados en el handshake.

* Decodifica el mensaje.

In [37]:
def receptor(datos_canal, codigos_objeto, codigos_ubicacion):

    objeto_codificado = datos_canal["Se detectó"]
    ubicacion_codificada = datos_canal["Ubicación"]
    imagenes = datos_canal["Imagenes"]

    handshake_codigos_objeto = pd.DataFrame(list(codigos_objeto.items()), columns=['Simbolo', 'Código'])
    handshake_codigos_ubicacion = pd.DataFrame(list(codigos_ubicacion.items()), columns=['Simbolo', 'Código'])

    # Hasheo los códigos tunstall de objeto y ubicación
    diccionario_hash_objeto = hashear_handshake(codigos_objeto)
    diccionario_hash_ubicacion = hashear_handshake(codigos_ubicacion)

    # Al handshake agregos los hash
    handshake_codigos_objeto['Hash'] = handshake_codigos_objeto['Código'].apply(lambda x: diccionario_hash_objeto[x])
    handshake_codigos_ubicacion['Hash'] = handshake_codigos_ubicacion['Código'].apply(lambda x: diccionario_hash_ubicacion[x])

    # Handshake
    print("Handshake de objeto en Tunstall con Hash:")
    print(handshake_codigos_objeto)
    print("\nHandshake de ubicación en Tunstall con Hash:")
    print(handshake_codigos_ubicacion)

    lista_hashes_objeto = list(zip(handshake_codigos_objeto['Código'], handshake_codigos_objeto['Hash']))
    lista_hashes_ubicacion = list(zip(handshake_codigos_ubicacion['Código'], handshake_codigos_ubicacion['Hash']))

    # búsqueda binaria para los hashes de objeto y ubi
    paquetes_hashes_objeto = objeto_codificado
    paquetes_hashes_ubicacion = ubicacion_codificada

    resultado_busqueda_objeto, codigos_encontrados_objeto = busqueda_binaria(lista_hashes_objeto, paquetes_hashes_objeto)
    resultado_busqueda_ubicacion, codigos_encontrados_ubicacion = busqueda_binaria(lista_hashes_ubicacion, paquetes_hashes_ubicacion)

    """ Reconctruyendo el mensaje de los paquetes codificados en tunstall para la decodificación"""
    # asigno los codigos encontrados
    objeto_codificado = codigos_encontrados_objeto
    ubicacion_codificada = codigos_encontrados_ubicacion

    # verifico si se encontraron todos los hash en la busqueda
    if resultado_busqueda_objeto:
        print("¡Se encontraron todos los hashes de objeto en los paquetes!")
    else:
        print("No se encontraron todos los hashes de objeto en los paquetes.")

    if resultado_busqueda_ubicacion:
        print("¡Se encontraron todos los hashes de ubicación en los paquetes!")
    else:
        print("No se encontraron todos los hashes de ubicación en los paquetes.")

    # Ya que recupere el mensaje original en código tunstall, procedo a la decodificación
    objeto_decodificado = decodificacion_tunstall(objeto_codificado, codigos_objeto)
    ubicacion_decodificada = decodificacion_tunstall(ubicacion_codificada, codigos_ubicacion)

    datos_decodificados = {
        "Se detectó": objeto_decodificado,
        "Ubicación": ubicacion_decodificada,
        "Imagenes": imagenes
    }

    return datos_decodificados

In [38]:
info_original = recopilar_datos()
# Guardo las funciones en variables
info_transmitida_hasheada, codigos_objeto, codigos_ubicacion = transmisor_de_la_nave_espacial(info_original)
datos_canal, tiempo_propagacions = fibra_optica(info_transmitida_hasheada)
info_receptada = receptor(datos_canal, codigos_objeto, codigos_ubicacion)
# Finalmente la información ha llegado a los científicos de la Estación Espacial


Paquetes codificados en Tunstall:
{'Se detectó': '0011 0111 1000 0011 1001 0001 1010 0000 0100 0000 0010 0101 0010 0110 0100 0101 0001 1011 0001 0010 0000 1100 0110 0000 1101 0000 1110', 'Ubicación': '0110 0001 0001 0111 0010 1000 0000 0100 0101 0000 0001 0011 0010 0011 0000 0100 0101 0000 1001 1010 0010 1011 0011 1100', 'Imagenes': ['imagen1.jpg', 'imagen2.jpg']}
Handshake de objeto en Tunstall con Hash:
   Simbolo Código                                               Hash
0        e   0000  9af15b336e6a9619928537df30b2e6a2376569fcf9d7e7...
1            0001  888b19a43b151683c87895f6211d9f8640f97bdc8ef32f...
2        r   0010  0b08e3dcc50fe4e5cee9b0b3a671a8db936f8335ba9050...
3        a   0011  a8d0b6f0939cfd883251f62b265f971ef8a5ab97eee32b...
4        t   0100  5e7b571a60a7c187d6a4cb8bbedbe4e69d4caa49b51d9d...
5        o   0101  07334386287751ba02a4588c1a0875dbd074a61bd9e6ab...
6        i   0110  a5c3dd48facf21ed5f916d0ae979091fead570e6aea6c1...
7        g   0111  5bedd317328c9ed79ec

In [39]:
print("===== Datos Recibidos en la Estación Espacial =====")
print("===== Perseverance 2.1 ha capturado los siguientes datos ===== \n")

print("Datos originales:")
print(info_original)
print()

print("Paquetes transmitidos:")
print(info_transmitida_hasheada)
print()

print("Paquetes transmitidos después del canal:")
print(datos_canal)
print()

print("Datos receptados:")
print(info_receptada)

===== Datos Recibidos en la Estación Espacial =====
===== Perseverance 2.1 ha capturado los siguientes datos ===== 

Datos originales:
{'Se detectó': 'agua, meterorito & relieves', 'Ubicación': (996.04, 491.14, 482.51), 'Imagenes': ['imagen1.jpg', 'imagen2.jpg']}

Paquetes transmitidos:
{'Se detectó': ['a8d0b6f0939cfd883251f62b265f971ef8a5ab97eee32b91460f08b965601d93', '5bedd317328c9ed79ecd3aecee74d37f97813cb4ce61ef3402eb388fb369fa86', '40510175845988f13f6162ed8526f0b09f73384467fa855e1e79b44a56562a58', 'a8d0b6f0939cfd883251f62b265f971ef8a5ab97eee32b91460f08b965601d93', 'fe675fe7aaee830b6fed09b64e034f84dcbdaeb429d9cccd4ebb90e15af8dd71', '888b19a43b151683c87895f6211d9f8640f97bdc8ef32f03dbe057c8f5e56d32', '7a5df5ffa0dec2228d90b8d0a0f1b0767b748b0a41314c123075b8289e4e053f', '9af15b336e6a9619928537df30b2e6a2376569fcf9d7e773eccede65606529a0', '5e7b571a60a7c187d6a4cb8bbedbe4e69d4caa49b51d9ddf3320afd793f146bf', '9af15b336e6a9619928537df30b2e6a2376569fcf9d7e773eccede65606529a0', '0b08e3dcc50fe4e